In [1]:
sc.version

'3.1.1'

### Imports

In [2]:
from configparser import ConfigParser
from pathlib import Path
import time


### Define directories

In [3]:
# Define directories
#
# Relevant directories are read from the config file:
# dir_data:    full path to hdfs directory where the raw data .gz files are stored
# dir_parquet: full path to hdfs directory where the parquet tables will be stored

cf = ConfigParser()
cf.read("../config.cf")

dir_data = Path(cf.get("spark", "dir_data"))
dir_parquet = Path(cf.get("spark", "dir_parquet"))

In [4]:
# Configuration hdfs
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
hdfs_dir_parquet = spark._jvm.org.apache.hadoop.fs.Path(dir_parquet.as_posix())

# Create output directories if they do not exist
# !hadoop dfs ...
# !hadoop dfs -put 20220201 /export/ml4ds/IntelComp/Datalake/SemanticScholar/

if not fs.exists(hdfs_dir_parquet):
    fs.mkdirs(hdfs_dir_parquet)


### Save files tables to parquet

In [9]:
tables = set([f.name.split("_")[0] for f in dir_data.iterdir() if f.name.endswith(".gz")])
# tables = ["tls803", "tls801"]

for t in tables:
    t0 = time.time()
    print("Table:", t)
    # Read all gz of same table
    df = spark.read.csv(f"file://{dir_data.joinpath(t).as_posix()}*.gz", header=True)
#     df.show(5, truncate=False)

    # Save to parquet
    df.write.parquet(
        dir_parquet.joinpath(f"{t}.parquet").as_posix(),
        mode="overwrite",
    )
    print(f"Dataframe saved in {time.time()-t0:.3f}s")

tls803


Dataframe saved in 14.163s
tls801


Dataframe saved in 3.141s
